In [1]:
# Basic utilities
import re
import glob
import sys
from datetime import date
from copy import deepcopy
from skimage.exposure import rescale_intensity
import matplotlib.pyplot as plt

# Data wrangling packages
import numpy as np
from scipy import stats
from skimage.filters import threshold_yen

# Cellpose - the current belle of the ball
from cellpose import models, io

# Grab utilities from the imaging directories dir
utilsDir = re.sub(r'Cell Segmentation', 'Imaging Utilities', str(sys.path[0]))

# Homebrew utilities for importing ims files & basic image manipulation
sys.path.append(utilsDir)
import ims_file_utils as ims
import image_processing_utils as impro
from skimage.segmentation import find_boundaries

from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import scipy.ndimage as ndimage
import matplotlib.pyplot as plt
from skimage import io
import glob
from math import sqrt
import pickle as pkl
import os, psutil
process = psutil.Process(os.getpid())
# Grab utilities from the imaging directories dir
utilsDir = re.sub(r'Registration', 'Imaging Utilities', str(sys.path[0]))
# Homebrew utilities for importing ims files & basic image manipulation
sys.path.append(utilsDir)
from functools import reduce
from scipy.spatial.distance import cdist, pdist
from skimage.exposure import rescale_intensity
from skimage.segmentation import find_boundaries

def spotcall(img, min_sig, max_sig, nsig, th):
    blobs_log = blob_log(img, min_sigma=min_sig, max_sigma=max_sig, num_sigma=nsig, threshold=th)
    # Compute radii in the 3rd column.
    blobs_log[:, 2] = blobs_log[:, 2] * sqrt(2)
    color = 'lime'
    title = 'Laplacian of Gaussian'
    #plt.rcParams['figure.figsize'] = [20, 15]
    #fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)
    #ax.set_title(title)
    #ax.imshow(img)
    #for blob in blobs_log:
    #    y, x, r = blob
    #    c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
    #    ax.add_patch(c)
    #ax.set_axis_off()
    #plt.tight_layout()
    #plt.show()
    return blobs_log

def numpyToList(points):
    # Takes a tuple of per-axis coordinates and converts it to a list of tuples
    outList = list()
    for i in range(len(points[0])):
        point = list()
        for axis in points:
            point.append(axis[i])
        outList.append(tuple(point))
    return(outList)

def fociDetection(image, masks_nuc, name, outFile):
    # Use laplacian of gaussian to detect blobs
    spots = spotcall(image,min_sig=0.05, max_sig=1,nsig=3,th=0.5) # Decrease threshold if spots are not identified and min sigma if spots are smaller
    spots = np.unique(spots, axis=0)
    print("Num " + name + " foci detected:", len(spots))

    # save spot coordinates
    with open(outFile + '_' + name + '_' + str(len(spots)) + '.pkl', 'wb') as filehandler:
        pkl.dump(spots, filehandler)
        
    return(spots)

# 1. Import registered antibody staining images

In [3]:
config = {
        "data_url" : "A1_1",
    }
print(config)

{'data_url': 'A1_1'}


In [4]:
runName = config['data_url']
opdir = '/mnt/disks/external/jg4159/20221215_DDR364xDrugs/' # Path to output
ip = opdir + "RegisteredImages_Ab_" + runName + ".pickle"
mk = opdir + "Unfiltered_segmentation_2D_"+ runName +".pickle"
print(ip)
print(mk)
print(runName)

/mnt/disks/external/jg4159/20221215_DDR364xDrugs/RegisteredImages_Ab_A1_1.pickle
/mnt/disks/external/jg4159/20221215_DDR364xDrugs/Unfiltered_segmentation_2D_A1_1.pickle
A1_1


In [5]:
import glob
#Loading ims files for different cycles
lof = glob.glob(mk)
filehandler = open(lof[0], 'rb')
masks_mem, masks_nuc, ref_mem = pkl.load(filehandler) # import segmentation
print(len(ref_mem))
filehandler.close()

25124


In [6]:
# rescaling
filehandler = open(ip, 'rb')
img_stack = pkl.load(filehandler)
filehandler.close()

for img in img_stack[1:]:
    for i in range(img.shape[0]):
        img[i] *= 65535
        img[i] = img[i].astype(np.uint16)
        m = np.median(img[i][img[i] > 0])
        print(np.percentile(img[i], (97.5, 99.8)))

[114. 127.]
[163. 216.]
[105. 121.]
[123. 140.]
[579. 772.]
[108. 112.]
[105. 106.]
[125. 149.]
[124. 159.]
[587. 782.]
[105. 107.]
[108. 111.]
[112. 117.]
[126. 135.]
[702. 933.]


# 2. DDR foci detection

In [8]:
#Ab0
EPCAM_p2 = 101
EPCAM_p98 = 120
EPCAM = rescale_intensity(img_stack[0][0], (EPCAM_p2, EPCAM_p98), (0, 255))

rad_p2 = 118
rad_p98 = 140
rad = rescale_intensity(img_stack[0][1], (rad_p2, rad_p98), (0, 255))

yH2AX_p2 = 114
yH2AX_p98 = 140
yH2AX = rescale_intensity(img_stack[0][2], (yH2AX_p2, yH2AX_p98), (0, 255))

RPA2_p2 = 114
RPA2_p98 = 140
RPA2 = rescale_intensity(img_stack[0][3], (RPA2_p2, RPA2_p98), (0, 255))


In [9]:
# # Sanity check for thresholding
# plt.rcParams['figure.figsize'] = [15*2, 15*15]
# f, axarr = plt.subplots(15,2)
# for i in range(3):
#     for j in range(8):
#         axarr[j][i].imshow(gamstack[3*j+i][500:1500,500:1500], cmap='gray')

In [11]:
start_time = time.time()
filename = opdir + runName + '_all_spots_'
rad51_spots = fociDetection(rad, masks_nuc, 'RAD51', filename + str(rad_p2) + '_' + str(rad_p98))
yh2ax_spots = fociDetection(yH2AX, masks_nuc, 'yH2AX', filename + str(yH2AX_p2) + '_' + str(yH2AX_p98))
rpa2_spots = fociDetection(RPA2, masks_nuc, 'RPA2', filename + str(RPA2_p2) + '_' + str(RPA2_p98))
print(f'--- Foci Detection: {(time.time()-start_time)} seconds')

Num RAD51 foci detected: 68914
Num yH2AX foci detected: 81454
Num RPA2 foci detected: 55606
--- Foci Detection: 110.18772053718567 seconds


# 3. Assign foci to cell masks

In [12]:
start_time = time.time()
# Strip the z axis off the detected foci coords and convert to integers
detectedFoci = [rad51_spots[:,0:2].astype('uint16'),
               yh2ax_spots[:,0:2].astype('uint16'), 
                rpa2_spots[:,0:2].astype('uint16')]
names = ('RAD51 spots', 'yH2AX spots', 'RPA2 spots')

# Create a dictionary where every key is a foci coordinate, and the value is the protein
spotMatch = {}
for i, spots in enumerate(detectedFoci):
    for spot in spots:
        if spotMatch.get((spot[0], spot[1])):
            spotMatch[(spot[0], spot[1])].append(names[i])
        else:
            spotMatch[(spot[0], spot[1])] = [names[i]]

# For each nuclei, see which spots in the membrane are indexed in the key 
for i, cellID in enumerate(ref_mem):
    if i %1000==0:
        print(i)
    for name in names:
        ref_mem[cellID][name] = 0
    
    # For each Nuclei Coordinate look for a spot
    nucCoord = numpyToList(ref_mem[cellID]['Nuclei Pixels 2D'])
    
    for coord in nucCoord:
        foci = spotMatch.get(coord)
        if foci:
            for focus in foci:
                ref_mem[cellID][focus] += 1
                
print(f'--- Foci Assignment: {(time.time()-start_time)} seconds')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
--- Foci Assignment: 7.1920318603515625 seconds


# 4. Detect micronuclei

In [13]:
import time
start_time = time.time()
for i, mem_id in enumerate(ref_mem):
    #print(mem_id)
    mem_pxls = ref_mem[mem_id]["Membrane Pixels 2D"]
    # Get the most common membrane mask value for that location
    nucs = masks_nuc[mem_pxls]
    if sum(nucs)>0: #nuc present
        nucs = nucs[nucs!=0] #non-zero nuc_ids
        #nuc_id = stats.mode(nucs)[0][0] # pick the mode as the nuc_id
        nuc_id = []
        for j in np.unique(nucs):
            #print(j)
            nuc_id.append(j)
        nuc_size = [len(np.where(nucs==x)[0]) for x in nuc_id]
        nuc_id_sorted = sorted(nuc_id, reverse=True, key=lambda x: nuc_size[nuc_id.index(x)])
        #nuc_pxls = [np.where(nucs==x)[0] for x in nuc_id_sorted]
        #nuc_pxls_2d = [np.where(masks_nuc==x) for x in nuc_id_sorted]
        
        ref_mem[mem_id]["nuc_id_sorted"] =  nuc_id_sorted
        #ref_mem[mem_id]["nuc_pxls_sorted"] = nuc_pxls
        ref_mem[mem_id]["small_nuc"] = sum([x>100 for x in nuc_size])
        #ref_mem[mem_id]["nuc_pxls_2d"] = nuc_pxls_2d
        #ref_mem[mem_id]["mem_pxls_2d"] = np.where(masks_mem==mem_id)
print("--- %s seconds ---" % (time.time() - start_time))

--- 6.035405397415161 seconds ---


In [14]:
nuc_p2 = 105
nuc_p98 = 225
nuc = rescale_intensity(img_stack[0][4], (nuc_p2,nuc_p98), (0, 255)) # DAPI channel
flat_dapi = nuc.ravel()
masks_nuc_cropped = masks_nuc[:nuc.shape[0],:nuc.shape[1]]
flat_nuc = masks_nuc_cropped.ravel()

flat_dapi[flat_nuc>0] = 0
# Remove cytoplamic background
flat_dapi[flat_dapi<=120] = 0

# dialate nuc masks by 1 pixel
import cv2
radius=2
outl = find_boundaries(masks_nuc, connectivity=1, mode='thick', background=0)
outl_mask = masks_nuc_cropped #Use this for outline with color scheme similar to the cell mask
#Creating a kernel to add pixels of desired size
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
#adding pixels to foreground -- will only work on binary images --- use following command to change thickness
thick_outl = cv2.dilate(outl_mask.astype(np.uint16), kernel, iterations=radius) #Vary iterations parameter for varying thickness 
flat_dapi[thick_outl.ravel()>0] = 0

# Use laplacian of gaussian to detect blobs
spots = spotcall(nuc, min_sig=1, max_sig=5,nsig=5,th=0.5) # Decrease threshold if spots are not identified and min sigma if spots are smaller
spots = np.unique(spots, axis=0)
print("Foci detected:", len(spots))

Foci detected: 106284


# 5. Assign micronuclei to cell masks

In [15]:
# assign micronuclei foci to cell mask
start_time = time.time()
# Create a dictionary where every key is a foci coordinate, and the value is the protein
spotMatch = {}
for spot in spots:
    if spotMatch.get((spot[0], spot[1])):
        spotMatch[(spot[0], spot[1])].append('micronuclei')
    else:
        spotMatch[(spot[0], spot[1])] = ['micronuclei']

# For each nuclei, see which spots in the membrane are indexed in the key 
for i, cellID in enumerate(ref_mem):
    if i %1000==0:
        print(i)
    ref_mem[cellID]['micronuclei'] = 0
    
    # For each Nuclei Coordinate look for a spot
    memCoord = numpyToList(ref_mem[cellID]['Membrane Pixels 2D'])
    
    for coord in memCoord:
        foci = spotMatch.get(coord)
        if foci:
            for focus in foci:
                ref_mem[cellID][focus] += 1
                
print(f'--- Foci Assignment: {(time.time()-start_time)} seconds')

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
--- Foci Assignment: 27.099990367889404 seconds


# 6. Output pickle & csv files

In [16]:
#Produce output pickle
with open(opdir + runName + "_segmentation_2D_foci_micronuclei" + '.pickle', 'wb') as filehandler:
    pkl.dump((masks_mem, masks_nuc, ref_mem), filehandler)

In [ ]:
data = []
for key in ref_mem.keys(): # use keys in the saved dict
    if True:
        data.append([key,
            ref_mem[key]['RAD51 spots'],
            ref_mem[key]['yH2AX spots'],
            ref_mem[key]['RPA2 spots'],
            ref_mem[key]['micronuclei'],
            len(ref_mem[key]['Nuclei Pixels']), 
            len(ref_mem[key]['Membrane Pixels'])])

df2 = pd.DataFrame(data, columns=['cell_id',
                            'RAD51_foci',
                            'yH2AX_foci',
                            'RPA2_foci',
                            "micronuclei_spots",
                            "nuclei_size", 
                            "mem_size"])

df2 = df2.sort_values(by="cell_id")
df2.to_csv(opdir + runName+"_foci_micronuclei_" + str(date.today()) + '.csv', index=False)